<a href="https://colab.research.google.com/github/Oussama-Aouina/Booky/blob/main/tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%writefile hello.c
#include <stdio.h>
#include <stdlib.h>


int main(){
    int n;
    printf("Entrez un nombre entier : ");
    scanf("%d", &n);
    printf("le nombre entré est:%d", n);
    return 0;
}

Overwriting hello.c


In [5]:
!nvcc hello.c -o hello

In [6]:
!./hello

Entrez un nombre entier : 6
le nombre entré est:6

In [10]:
%%writefile hello.cu
#include <stdio.h>
#include <stdlib.h>


__global__ void main(){
    int n;
    printf("Entrez un nombre entier : ");
    scanf("%d", &n);
    printf("le nombre entré est:%d", n);
    return 0;
}

Overwriting hello.cu


In [11]:
!nvcc hello.cu -o hello2

hello.cu(5): warning #951-D: return type of function "main" must be "int"
  __attribute__((global)) void main(){
                               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

hello.cu(5): error: function main cannot be marked __device__ or __global__
  __attribute__((global)) void main(){
                 ^

hello.cu(10): error: return value type does not match the function type
      return 0;
             ^

2 errors detected in the compilation of "hello.cu".


In [9]:
!./hello2

Entrez un nombre entier : 4
le nombre entré est:4

In [24]:
%%writefile hello.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

__global__ void affiche(){

    printf("bonjour");

}
int main (void){
    affiche <<<2,2>>>(); //grid.x -> le numero de thread / grid.y -> le numero de bloc
    cudaDeviceSynchronize(); // synchronisation entre le cpu et le gpu
    return 0;
}


Overwriting hello.cu


In [25]:
!nvcc hello.cu -o hello3

In [26]:
!./hello3

bonjourbonjourbonjourbonjour

In [31]:
%%writefile ex2.c
#include <cuda.h>
#include <device_launch_parameters.h>
#include <cuda_device_runtime_api.h>
#include <cuda_runtime_api.h>
#include <stdio.h>
#include<stdlib.h>
#define N 12


int add(int *a, int *b, int *c)
{
    for (int i=0;i<N;i++)
    c[i] = a[i] + b[i];

 }

void charger(int* a)
{
   int i;
   for (i = 0; i < N; ++i)
    a[i] = i;
}


int main(void)
{
    int *a, *b, *c;  // host copies of a, b, c

    a = (int *)malloc(N*sizeof(int));
    charger(a);
    b = (int *)malloc(N*sizeof(int));
    charger(b);
    c = (int *)malloc(N*sizeof(int));

    add(a,b,c);

    for (int i=0;i<N;i++)
      printf("%d |", c[i]);
      //cleanup
      free(a);
      free(b);
      free(c);

 return 0;
}

Overwriting ex2.c


In [32]:
!nvcc ex2.c -o ex2

In [33]:
!./ex2

0 |2 |4 |6 |8 |10 |12 |14 |16 |18 |20 |22 |

In [1]:
%%writefile ex2.cu
#include <cuda.h>
#include <device_launch_parameters.h>
#include <cuda_device_runtime_api.h>
#include <cuda_runtime_api.h>
#include <stdio.h>
#include<stdlib.h>
#define N 12
//using namespace std;

__global__ void add(int *a, int *b, int *c)
{
    //for (int i=0;i<N;i++)
    c[blockIdx.x] = a[blockIdx.x] + b[blockIdx.x];//pour le bloc thread on peut ecrire threadIdx a la place de blockIdx.x
 }

void charger(int* a)
{
   int i;
   for (i = 0; i < N; ++i)
    a[i] = i;
}


int main(void)
{
    int *a, *b, *c;  // host(CPU) copies of a, b, c
    int *d_a,*d_b,*d_c; // device(GPU) copies of a, b, c
    int size=N*sizeof(int);
    // alloc spaces for device copies of a b and c
    cudaMalloc((void**)&d_a,size);
    cudaMalloc((void**)&d_b,size);
    cudaMalloc((void**)&d_c,size);
    a = (int *)malloc(N*sizeof(int));
    charger(a);
    b = (int *)malloc(N*sizeof(int));
    charger(b);
    c = (int *)malloc(N*sizeof(int));
    // copy inputs from host to device
    cudaMemcpy(d_a,a,size,cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,b,size,cudaMemcpyHostToDevice);

    add<<<N,1>>>(d_a,d_b,d_c);
    // copy output from device to host
    cudaMemcpy(c,d_c,size,cudaMemcpyDeviceToHost);

    for (int i=0;i<N;i++)
      printf("%d |", c[i]);

    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

 return 0;
}

Writing ex2.cu


In [2]:
!nvcc ex2.cu -o ex2cu

In [3]:
!./ex2cu

0 |2 |4 |6 |8 |10 |12 |14 |16 |18 |20 |22 |